# Working with data

In [13]:
%pip install torch

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.3.2 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [14]:
%pip install torchvision

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.3.2 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [15]:
import torch
import pandas as pd
from torch import nn
from torch.utils.data import DataLoader, TensorDataset
from torchvision import datasets
from torchvision.transforms import ToTensor
from torch import Tensor

In [17]:
# Load the MNIST dataset
train_dataset = datasets.MNIST(root='./data', train=True, download=True, transform=ToTensor())
test_dataset = datasets.MNIST(root='./data', train=False, download=True, transform=ToTensor())

# Print the size of the training and test datasets
print("Size of the training dataset:", len(train_dataset))
print("Size of the test dataset:", len(test_dataset))

# Define batch size
batch_size = 64

# Create DataLoader instances for Train_data and Test_data
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=True)

100%|██████████| 9912422/9912422 [00:01<00:00, 5132236.78it/s]


Extracting ./data\MNIST\raw\train-images-idx3-ubyte.gz to ./data\MNIST\raw



100%|██████████| 28881/28881 [00:00<00:00, 3735182.20it/s]


Extracting ./data\MNIST\raw\train-labels-idx1-ubyte.gz to ./data\MNIST\raw



100%|██████████| 1648877/1648877 [00:01<00:00, 1241540.37it/s]


Extracting ./data\MNIST\raw\t10k-images-idx3-ubyte.gz to ./data\MNIST\raw



100%|██████████| 4542/4542 [00:00<?, ?it/s]

Extracting ./data\MNIST\raw\t10k-labels-idx1-ubyte.gz to ./data\MNIST\raw

Size of the training dataset: 60000
Size of the test dataset: 10000


In [20]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

In [21]:
model = NeuralNetwork()
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


In [22]:
learning_rate = 1e-3
batch_size = 198
epochs = 9
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

In [23]:
def train_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    # Set the model to training mode - important for batch normalization and dropout layers
    # Unnecessary in this situation but added for best practices
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        # Compute prediction and loss
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        if batch % 100 == 0:
            loss, current = loss.item(), batch * batch_size + len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")


def test_loop(dataloader, model, loss_fn):
    # Set the model to evaluation mode - important for batch normalization and dropout layers
    # Unnecessary in this situation but added for best practices
    model.eval()
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss, correct = 0, 0

    # Evaluating the model with torch.no_grad() ensures that no gradients are computed during test mode
    # also serves to reduce unnecessary gradient computations and memory usage for tensors with requires_grad=True
    with torch.no_grad():
        for X, y in dataloader:
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()

    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [24]:
for name, param in model.named_parameters():
    print(f"Layer: {name} | Size: {param.size()} | Values : {param[:2]} \n")

Layer: linear_relu_stack.0.weight | Size: torch.Size([512, 784]) | Values : tensor([[ 0.0355,  0.0286,  0.0282,  ..., -0.0248,  0.0182,  0.0184],
        [-0.0022,  0.0244, -0.0228,  ...,  0.0124,  0.0181,  0.0021]],
       grad_fn=<SliceBackward0>) 

Layer: linear_relu_stack.0.bias | Size: torch.Size([512]) | Values : tensor([ 0.0232, -0.0324], grad_fn=<SliceBackward0>) 

Layer: linear_relu_stack.2.weight | Size: torch.Size([512, 512]) | Values : tensor([[ 0.0009, -0.0105,  0.0273,  ...,  0.0096,  0.0176, -0.0240],
        [ 0.0362, -0.0319,  0.0143,  ...,  0.0222, -0.0129, -0.0317]],
       grad_fn=<SliceBackward0>) 

Layer: linear_relu_stack.2.bias | Size: torch.Size([512]) | Values : tensor([ 0.0161, -0.0421], grad_fn=<SliceBackward0>) 

Layer: linear_relu_stack.4.weight | Size: torch.Size([10, 512]) | Values : tensor([[-0.0430, -0.0124, -0.0026,  ...,  0.0155,  0.0339,  0.0398],
        [-0.0159,  0.0196,  0.0355,  ...,  0.0174,  0.0054,  0.0417]],
       grad_fn=<SliceBackward0>)

In [25]:
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train_loop(train_loader, model, loss_fn, optimizer)
    test_loop(test_loader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 2.298172  [   64/60000]
loss: 2.300147  [19864/60000]
loss: 2.292028  [39664/60000]
loss: 2.290482  [59464/60000]
loss: 2.289212  [79264/60000]
loss: 2.274324  [99064/60000]
loss: 2.280329  [118864/60000]
loss: 2.267577  [138664/60000]
loss: 2.261220  [158464/60000]
loss: 2.262212  [178264/60000]
Test Error: 
 Accuracy: 36.1%, Avg loss: 2.256245 

Epoch 2
-------------------------------
loss: 2.256849  [   64/60000]
loss: 2.248371  [19864/60000]
loss: 2.248644  [39664/60000]
loss: 2.248408  [59464/60000]
loss: 2.232877  [79264/60000]
loss: 2.224309  [99064/60000]
loss: 2.207520  [118864/60000]
loss: 2.207941  [138664/60000]
loss: 2.218373  [158464/60000]
loss: 2.185412  [178264/60000]
Test Error: 
 Accuracy: 61.6%, Avg loss: 2.192158 

Epoch 3
-------------------------------
loss: 2.186047  [   64/60000]
loss: 2.193542  [19864/60000]
loss: 2.179851  [39664/60000]
loss: 2.154667  [59464/60000]
loss: 2.164060  [79264/60000]
loss: 2.166482  [9

In [26]:
torch.save(model.state_dict(), 'model.pt')